<b>Dream Team Optimization (Session 10)</b>

In [1]:
# typical BUAD 313 import:
import numpy as np
from gurobipy import GRB, Model, quicksum

In [2]:
# define the model m labeled "Dream Team"
m = Model('Dream Team')

# DATA 

# define a list of players, numbered 1 through 20
players = range(1, 21)


Rebounds = [1, 2, 3, 4, 5, 7, 7, 4, 8, 5, 10, 8, 10, 9, 6, 16, 11, 12, 11, 9]
Assists = [7, 14, 12, 4, 9, 6, 8, 2, 2, 5, 6, 8, 2, 5, 3, 2, 1, 5, 1, 1]
Points = [10, 14, 19, 18, 20, 21, 23, 13, 17, 25, 20, 30, 24, 15, 17, 3, 27, 26, 21, 14]
Defense = [10, 9, 8, 6, 8, 10, 10, 5, 8, 8, 9, 10, 9, 7, 6, 6, 9, 10, 9, 8]
Heights = [71, 72, 76, 72, 75, 77, 80, 77, 82, 76, 82, 81, 87, 82, 82, 81, 88, 86, 87, 84]


### Dictionary versions are all lower-case
# create a dictionary of the rebound data indexed by player
rebounds = dict(zip(players, Rebounds))

# create a dictionary of the assist data indexed by player
assists = dict(zip(players, Assists))

# create a dictionary of the points data indexed by player
points = dict(zip(players, Points))

# create a dictionary of the defense data indexed by player
defense = dict(zip(players, Defense))

# create a dictionary of the height data indexed by player
heights = dict(zip(players, Heights))

# print a table of rebounds, assists, heights, points, and defense by player. specifically, each row should be one player and the columns should be tab spaced. abbreviate the column names as needed to fit the table in the space provided.
print("Player\tReb\tAst\tPts\tDef\tHt")
for p in players:
    print(f"{p}\t{rebounds[p]}\t{assists[p]}\t{points[p]}\t{defense[p]}\t{heights[p]}")

Set parameter Username
Set parameter LicenseID to value 2609347
Academic license - for non-commercial use only - expires 2026-01-13
Player	Reb	Ast	Pts	Def	Ht
1	1	7	10	10	71
2	2	14	14	9	72
3	3	12	19	8	76
4	4	4	18	6	72
5	5	9	20	8	75
6	7	6	21	10	77
7	7	8	23	10	80
8	4	2	13	5	77
9	8	2	17	8	82
10	5	5	25	8	76
11	10	6	20	9	82
12	8	8	30	10	81
13	10	2	24	9	87
14	9	5	15	7	82
15	6	3	17	6	82
16	16	2	3	6	81
17	11	1	27	9	88
18	12	5	26	10	86
19	11	1	21	9	87
20	9	1	14	8	84


In [3]:
# create a gurobi model called dream team
m = Model('Dream Team')

# create a binary variable for each player
x = m.addVars(players, vtype=GRB.BINARY, name='x')

##VG I was fed up with Copilot today, so I decided to write this code by myself!
#it's not hard and you might try too!
m.addConstr(quicksum(x[p] for p in players) == 12, name='team_size')
m.addConstr(quicksum(x[p] * points[p] for p in players) >= 18 * 12, name='points_per_game')
m.addConstr(quicksum(x[p] * rebounds[p] for p in players) >= 7 * 12, name='rebounds_per_game')
m.addConstr(quicksum(x[p] * assists[p] for p in players) >= 6 * 12, name='assists_per_game')
m.addConstr(quicksum(x[p] * defense[p] for p in players) >= 8.5 * 12, name='defense_rating')
m.addConstr(quicksum(x[p] * heights[p] for p in players) >= 79 * 12, name='height')


#Positional depth
#remember, range (start, stop) includes start but not stop
m.addConstr(quicksum(x[p] for p in range(1, 6)) >= 3, name='pg_depth')
m.addConstr(quicksum(x[p] for p in range(4, 12)) >= 4, name='sg_depth')
m.addConstr(quicksum(x[p] for p in range(9, 17)) >= 4, name='sf_depth')
m.addConstr(quicksum(x[p] for p in range(16, 21)) >= 3, name='pf_depth')


#NCAA
m.addConstr(x[4] + x[8] + x[15] + x[20] >= 2, name='NCAA')

#Lakers
m.addConstr(x[1] + x[7] + x[12] + x[16] <= 3, name='Lakers')

#player idiosyncracies
m.addConstr(x[9] + x[5] <= 1, name='player_idiosyncracies')
m.addConstr(x[2] == x[19])  

m.setObjective(quicksum(x[p] * points[p] for p in players)/12, GRB.MAXIMIZE)

m.optimize()




Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14 rows, 20 columns and 158 nonzeros
Model fingerprint: 0xcf417bab
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [2e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Found heuristic solution: objective 20.1666667
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 13 rows, 19 columns, 144 nonzeros
Variable types: 0 continuous, 19 integer (19 binary)

Root relaxation: objective 2.200000e+01, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      22.00

In [4]:
#print the solution 
print("Solution:")
for p in players:
    if x[p].x > 0.5:
        print(f"Player {p} is on the team")

Solution:
Player 2 is on the team
Player 3 is on the team
Player 4 is on the team
Player 5 is on the team
Player 7 is on the team
Player 10 is on the team
Player 12 is on the team
Player 13 is on the team
Player 15 is on the team
Player 17 is on the team
Player 18 is on the team
Player 19 is on the team


In [5]:
#Finding the second best solution.

xstar = m.getAttr('x', x)

m.addConstr(
    quicksum(x[p] for p in players if xstar[p] < 0.5) + 
    quicksum((1-x[p]) for p in players if xstar[p] > 0.5)     
    >= 1, name='dont pick xstar'
)

m.optimize()
print("Solution:")
for p in players:
    if x[p].x > 0.5:
        print(f"Player {p} is on the team")
# print the objective value
print(f"Objective value: {m.objVal}")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15 rows, 20 columns and 178 nonzeros
Model fingerprint: 0x589bd0ab
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [2e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint dont_pick_xstar by 1.000000000

Found heuristic solution: objective 20.1666667
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 14 rows, 19 columns, 162 nonzeros
Variable types: 0 continuous, 19 integer (19 binary)

Root relaxation: objective 2.197778e+01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    | 